In [10]:
import os
from PIL import Image
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter, A4
from reportlab.lib.units import inch

In [11]:

def create_pdf_from_screenshots(input_folder, output_pdf, columns=2, margin=0.5, space=0.1):
    # Get all image files from the input folder
    image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif'))]
    
    # Sort files to ensure consistent ordering
    image_files.sort()
    
    # Create a new PDF
    c = canvas.Canvas(output_pdf, pagesize=A4)
    width, height = A4
    
    # Calculate usable width and height
    usable_width = width - 2 * margin * inch
    usable_height = height - 2 * margin * inch
    
    # Calculate column width
    column_width = usable_width / columns
    
    x, y = margin * inch, height - margin * inch
    current_column = 0
    
    for image_file in image_files:
        img = Image.open(os.path.join(input_folder, image_file))
        img_width, img_height = img.size
        
        # Scale image to fit column width
        scale_factor = (column_width - space) / img_width
        new_height = img_height * scale_factor
        
        # Check if image fits on current page
        if new_height < usable_height:
            if y - new_height < margin * inch:
                # Move to next column or new page
                current_column += 1
                if current_column >= columns:
                    c.showPage()
                    current_column = 0
                    x = margin * inch
                    y = height - margin * inch
                else:
                    x += column_width + space
                    y = height - margin * inch
        else:
            print("[ERROR] Image does not fit")
        
        # Draw image
        c.drawImage(os.path.join(input_folder, image_file), x, y - new_height, width=(column_width - space), height=new_height)
        y -= new_height
    
    # Save the PDF
    c.save()

In [ ]:
input_folder = '../img'
output_pdf = "../merged.pdf"
create_pdf_from_screenshots(input_folder, output_pdf, columns=3, margin=0.3, space=10)